## Prior Plot

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_6/mash_plots.ipynb prior_plot \
    --prior_file ~/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/MASH_6_celltypes_promoter_udr_miss/Ast_Exc_Inh_Mic_OPC_Oli_pQTL_MiGA.with.na.promoter.udr.EZ.priorfit2.rds

In [1]:
# Prior plot with sharing pattern
[prior_plot]
parameter: per_chunk = '100'
parameter: prior_file = path
parameter: tol = '1E-6'
parameter: r = '0.32'
parameter: cwd = path('./')
input: prior_file
output: f"{_input:annn}.prior.pdf"
task: trunk_workers = 1, walltime = '20h', trunk_size = 1, mem = '20G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
        library("reshape2")
        library("ggplot2")
        library("gridExtra")
        library("tidyverse")
        dat = list()
        plot_sharing = function(X, col = 'black', to_cor=FALSE, title="", remove_names=F) {
                clrs <- colorRampPalette(rev(c("#D73027","#FC8D59","#FEE090","#FFFFBF",
                    "#E0F3F8","#91BFDB","#4575B4")))(128)
                if (to_cor) lat <- cov2cor(X)
                else lat = X/max(diag(X))
                lat[lower.tri(lat)] <- NA
                n <- nrow(lat)
                if (remove_names) {
                    colnames(lat) = paste('t',1:n, sep = '')
                    rownames(lat) = paste('t',1:n, sep = '')
                }
                melted_cormat <- melt(lat[n:1,], na.rm = TRUE)
                p = ggplot(data = melted_cormat, aes(Var2, Var1, fill = value))+
                    geom_tile(color = "white")+ggtitle(title) + 
                    scale_fill_gradientn(colors = clrs, limit = c(-1,1), space = "Lab") +
                    theme_minimal()+ 
                    coord_fixed() +
                    theme(axis.title.x = element_blank(),
                        axis.title.y = element_blank(),
                        axis.text.x = element_text(color=col, size=8,angle=45,hjust=1),
                        axis.text.y = element_text(color=rev(col), size=8),
                        title =element_text(size=10),
                        # panel.grid.major = element_blank(),
                        panel.border = element_blank(),
                        panel.background = element_blank(),
                        axis.ticks = element_blank(),
                        legend.justification = c(1, 0),
                        legend.position = c(0.6, 0),
                        legend.direction = "horizontal")+
                    guides(fill = guide_colorbar(title="", barwidth = 7, barheight = 1,
                        title.position = "top", title.hjust = 0.5))
                if(remove_names){
                    p = p + scale_x_discrete(labels= 1:n) + scale_y_discrete(labels= n:1)
                }
                return(p)
            }
        calculate_proportion = function(X, col = 'black', to_cor=FALSE, title="", remove_names=F) {
                if (to_cor) lat <- cov2cor(X)
                else lat = X/max(diag(X))
                lat[lower.tri(lat)] <- NA
                
                return(lat)
            }
        dat = readRDS('${_input}')
        meta = data.frame(names(dat$U), dat$w, stringsAsFactors=F)
        colnames(meta) = c("U", "w")
        tol = '${tol}' %>% as.numeric
        n_comp = length(meta$U[which(meta$w>tol)])
        meta = head(meta[order(meta[,2], decreasing = T),], nrow(meta))
        message(paste(n_comp, "components out of", length(dat$w), "total components have weight greater than", tol))
        res = list()
        res2 = list()

        for (i in 1:n_comp) {
            title = paste(meta$U[i], "w =", round(meta$w[i], 6))
            ##Handle updated udr data structure
            if(is.list(dat$U[[meta$U[i]]])){
            res[[i]] = plot_sharing(dat$U[[meta$U[i]]]$mat, to_cor = F, title=title, remove_names = FALSE)
                #calculate singleton/doubleton
                res2[[i]] = calculate_proportion(dat$U[[meta$U[i]]]$mat, to_cor = F, title=title, remove_names = FALSE)
                names(res2)[[i]]<-meta$U[[i]]
            } else if(is.matrix(dat$U[[meta$U[i]]])){
            res[[i]] = plot_sharing(dat$U[[meta$U[i]]], to_cor = F, title=title, remove_names = FALSE)
                #calculate singleton/doubleton
                res2[[i]] = calculate_proportion(dat$U[[meta$U[i]]], to_cor = F, title=title, remove_names = FALSE)
                names(res2)[[i]]<-meta$U[[i]]
            }
        }
        unit = 3.5
        n_col = 6
        n_row = ceiling(n_comp / n_col)

        res3<-ls()
        for(i in 1:length(res2)){
            res2[[i]][res2[[i]]< as.numeric('${r}')]=0
            res2[[i]][is.na(res2[[i]])]=0
            res3[[i]]<-sum(res2[[i]]>0)
            names(res3)[[i]]<-names(res2)[[i]]
        }
        singleton= round(sum(dat$w[names(res3)[res3==1]]),4)
        doubleton= round(sum(dat$w[names(res3)[res3==2]]),4)
        #save pdf version
        pdf("${_output}", width = unit * n_col, height = unit * n_row)
        do.call(gridExtra::grid.arrange, c(res, list(ncol = n_col, nrow = n_row,bottom = paste0("singleton = ",singleton," ; doubleton = ", doubleton))))
        dev.off()
        #save png version, which is convinent for preview in notebook
        Cairo::CairoPNG( 
        filename = gsub(".pdf",".png", "${_output}"), , width = unit * n_col, height = unit * n_row,units = "in",dpi = 300)           
        do.call(gridExtra::grid.arrange, c(res, list(ncol = n_col, nrow = n_row, bottom = paste0("singleton = ",singleton," ; doubleton = ", doubleton))))
        dev.off() 

## Posterior contrast 

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_6/mash_plots.ipynb posterior_contrast_plot \
    --cwd /mnt/vast/hpc/csg/rf2872/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/MASH_6_celltypes_udr_miss/contrast_merge

In [1]:
# plot contrast result
[posterior_cntrast_plot]
parameter: cwd = path('./')
input: f"{cwd}/posterior_sum.csv"
output: f"{cwd}/posterior_sum.png"
task: trunk_workers = 1, trunk_size = job_size, walltime = '24h',  mem = '10G', tags = f'{_output:bn}'  

R: expand = "${ }",stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout' 
        library(dplyr)
        library(tidyverse)
        library(ggnewscale)
        df <- read.csv("${_input:a}",row.names=1)
        colnames(df)<-gsub("DLPFC_","",colnames(df))
        for (i in 1:ncol(df)) {
            con1 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = T)[, 1]
            con2 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = T)[, 2]

            if (con1 > con2) {
                new.name <- paste0(con2, "_vs_", con1)
                colnames(df)[i] <- new.name
            }
        }

        ## summarizxse with approach 1: snp-feature pair
        snp.ratio <- df["n_sig_snp", ] / df["n_snp", ]
        snp.ratio <- snp.ratio %>%
            t() %>%
            as.data.frame()
        colnames(snp.ratio) <- "ratio"
        snp.ratio$group <- "snp"

        ## summarize with approach 2: feature
        fet.ratio <- df["n_sig_feature", ] / df["n_all_feature", ]
        fet.ratio <- fet.ratio %>%
            t() %>%
            as.data.frame()
        rownames(fet.ratio) <- paste0(stringr::str_split(rownames(fet.ratio), "_vs_", simplify = T)[, 2], "_vs_", stringr::str_split(rownames(fet.ratio), "_vs_", simplify = T)[, 1])
        colnames(fet.ratio) <- "ratio"
        fet.ratio$group <- "feature"
        ratio <- rbind(snp.ratio, fet.ratio)

        ## I need to add the below to make it Simmetrie

        cons <- rownames(ratio) %>%
            str_split(., "_vs_", simplify = T) %>%
            .[, 1] %>%
            unique()
        for (i in 1:length(cons)) {
            new.name <- paste0(cons[i], "_vs_", cons[i])
            ratio[new.name, ] <- 0
        }

        ratio$con1 <- stringr::str_split(rownames(ratio), "_vs_", simplify = T)[, 1]
        ratio$con2 <- stringr::str_split(rownames(ratio), "_vs_", simplify = T)[, 2]

        ## prepare for the plot, score1 is for snp-feature pair, score2 is for feature only
        ratio$score1 <- ratio$score2 <- 0
        ratio$score1[ratio$group == "snp"] <- ratio$ratio[ratio$group == "snp"]
        ratio$score2[ratio$group == "feature"] <- ratio$ratio[ratio$group == "feature"]
        ratio$label <- paste0(round(ratio$ratio, 4) * 100, "%")
        ratio$label[ratio$group == 0] <- NA

        # plot
        num_cols <- length(cons)
        height <- width <- 4 + num_cols * 0.5 
        ggplot(ratio[ratio$group == "snp", ], aes(x = con1, y = con2)) +
            geom_tile(aes(fill = score1)) +
            scale_fill_gradient2("SNP_Feature pair",
                low = "#762A83", mid = "white", high = "#1B7837"
            ) +
            new_scale("fill") +
            geom_tile(aes(fill = score2), data = subset(ratio, group != "snp")) +
            scale_fill_gradient2("Feature",
                low = "#1B7837", mid = "white", high = "#762A83"
            ) +
            geom_text(data = ratio, aes(label = label)) +
            theme_bw()
        #geom_text(data=ratio, aes(label = label, color = factor(group))) +theme_bw()
        #ggsave(gsub(".csv",".png",filename))

        ggsave("${_output}",width = width, height = height)
        ggsave(gsub(".pdf",".png","${_output}"),width = width, height = height)

## Posterior contrast with Fisher's test

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/rf2872/codes/CUMC_fungun_xqtl_analysis/analysis/Daily_documents/drafts/2023_6/mash_plots.ipynb posterior_contrast_fisher_plot \
    --cwd /mnt/vast/hpc/csg/rf2872/Work/Multivariate/MASH/From_SuSiE/2023.5_new/ep_MiGA/MASH_6_celltypes_udr_miss/  \
    --contrast_dir contrast_merge

In [2]:
# Prior plot with sharing pattern
[posterior_contrast_fisher_plot]
parameter: contrast_dir = "contrast"
parameter: group1 = "nonAD"
parameter: group2 = "AD"
parameter: sig_type = "feature"
parameter: cwd = path('./')
ad = file_target(f"{cwd:a}/{contrast_dir}/posterior_sum.csv")
non_ad = file_target(f"{cwd:a}/{contrast_dir}_non/posterior_sum.csv")
input: ad, non_ad
output: f"{cwd}/{contrast_dir}/posterior_sum_AD_fisher.pdf"
task: trunk_workers = 1, walltime = '20h', trunk_size = 1, mem = '20G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
        library(tidyverse)
        library(ggnewscale)
        #' Plot Contrast
        #'
        #' This function plots the contrast between two groups based on the provided data files.
        #'
        #' @param file1 The data file for group1.
        #' @param file2 The data file for group2.
        #' @param group1 The name of group1.
        #' @param group2 The name of group2.
        #' @param sig_type The type of significance (snp or feature).
        #' @param fisher_test A logical value indicating whether to perform Fisher's test (default: TRUE). If TRUE, plot will show the Fisher and group2 results. If FALSE, the plot will show the group1 and group2 results.
        #'
        #' @return A ggplot object representing the contrast plot.
        #'
        #' @examples
        #' plot_contrast(file1, file2, "Group1", "Group2", "snp", TRUE)
        #'
        plot_contrast <- function(file1, file2, group1, group2, sig_type, fisher_test=TRUE) {


            # Fisher test
            fisher_result <- matrix(ncol = 1, nrow = ncol(file1)) %>% as.data.frame()
            rownames(fisher_result) <- colnames(file1)

            if (sig_type == "snp") {
                sig <- "n_sig_feature"
                all <- "n_all_feature"
            }
            if (sig_type == "feature") {
                sig <- "n_sig_feature"
                all <- "n_all_feature"
            }

            # Prepare the data for plotting
            recolname_plot_df <- function(df) {
                for (i in 1:ncol(df)) {
                    con1 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = TRUE)[, 1]
                    con2 <- stringr::str_split(colnames(df)[i], "_vs_", simplify = TRUE)[, 2]

                    if (con1 > con2) {
                        new.name <- paste0(con2, "_vs_", con1)
                        colnames(df)[i] <- new.name
                    }
                }
                return(df)
            }

            
            colnames(file1) <- gsub("DLPFC_", "", colnames(file1))
            file1 <- recolname_plot_df(file1)
            colnames(file2) <- gsub("DLPFC_", "", colnames(file2))
            file2 <- recolname_plot_df(file2)

            ##prepare group1, could be nonAD data or fisher test, depends on the parameter
            
            if (fisher_test) {
            # Calculate fisher test
                # Fisher test
                fisher <- matrix(ncol = 1, nrow = ncol(file1)) %>% as.data.frame()
                rownames(fisher) <- colnames(file1)

                for (n in 1:length(colnames(file1))) {
                    col <- colnames(file1)[n]
                    # build a 2*2 matrix
                    data <- matrix(c(file1[sig, col], file2[sig, col], file1[all, col], file2[all, col]), nrow = 2)
                    colnames(data) <- c("sig", "all")
                    rownames(data) <- c(group1, group2)

                    # Fisher's test
                    result <- fisher.test(data)
                    fisher[col, ] <- result$p.value
                }

                rownames(fisher) <- gsub("DLPFC_", "", rownames(fisher))
                fisher <- fisher %>%
                    t(.) %>%
                    recolname_plot_df(.)
                group1.ratio <- -log10(fisher)
            } else {
                group1.ratio <- file1[sig, ] / file1[all, ]
            }
            group1.ratio <- group1.ratio %>%
                t() %>%
                as.data.frame()
            colnames(group1.ratio) <- "ratio"
            group1.ratio$group <- group1
            
            ##prepare group2, could be AD genes contrast proportion here
            group2.ratio <- file2[sig, ] / file2[all, ]
            group2.ratio <- group2.ratio %>%
                t() %>%
                as.data.frame()
            rownames(group2.ratio) <- paste0(stringr::str_split(rownames(group2.ratio), "_vs_", simplify = TRUE)[, 2], "_vs_", stringr::str_split(rownames(group2.ratio), "_vs_", simplify = TRUE)[, 1])
            colnames(group2.ratio) <- "ratio"
            group2.ratio$group <- group2

            ratio <- rbind(group1.ratio, group2.ratio)

            cons <- rownames(ratio) %>%
                str_split(., "_vs_", simplify = TRUE) %>%
                .[, 1] %>%
                unique()
            for (i in 1:length(cons)) {
                new.name <- paste0(cons[i], "_vs_", cons[i])
                ratio[new.name, ] <- 0
            }
            ratio$con1 <- stringr::str_split(rownames(ratio), "_vs_", simplify = TRUE)[, 1]
            ratio$con2 <- stringr::str_split(rownames(ratio), "_vs_", simplify = TRUE)[, 2]

            ratio$score1 <- ratio$score2 <- 0
            ratio$score1[ratio$group == group1] <- ratio$ratio[ratio$group == group1]
            ratio$score2[ratio$group == group2] <- ratio$ratio[ratio$group == group2]
            ratio$label <- paste0(round(ratio$ratio, 4) * 100, "%")
            if (fisher_test) {ratio$label[ratio$group == group1] <- round(ratio$ratio[ratio$group == group1], 2)}
            ratio$label[ratio$group == 0] <- NA

            # Plotting
            p <- ggplot(ratio[ratio$group == group1, ], aes(x = con1, y = con2)) +
                geom_tile(aes(fill = score1)) +
                scale_fill_gradient2(
                    if(fisher_test){"Fisher's -log10P"} else {group1},
                    low = "#762A83", mid = "white", high = "#1B7837"
                ) +
                new_scale("fill") +
                geom_tile(aes(fill = score2), data = subset(ratio, group != group1)) +
                scale_fill_gradient2(group2,
                    low = "red", mid = "white", high = "#762A83"
                ) +
                geom_text(data = ratio, aes(label = label)) +
                theme_bw()+ggtitle(sig_type)
            return(p)
        }


        ad <- read.csv('${ad}', row.names = 1)
        non.ad <- read.csv('${non_ad}', row.names = 1)

        plot_contrast(file1 = non.ad, file2 = ad,group1 = '${group1}',group2='${group2}',sig_type = '${sig_type}',fisher_test = T)
        cons <- colnames(ad) %>%
            str_split(., "_vs_", simplify = T) %>%
            .[, 1] %>%
            unique()
        num_cols <- length(cons)
        height <- width <- 4 + num_cols * 0.5 
        ggsave("${_output}",width = width, height = height)
        ggsave(gsub(".pdf",".png","${_output}"),width = width, height = height)

